In [ ]:
!pip -q install langchain huggingface_hub openai google-search-results tiktoken cohere
!pip install arxiv PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 61.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 6.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""

In [ ]:
!pip show langchain

Name: langchain
Version: 0.0.201
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, langchainplus-sdk, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


# Simple chain system

1. map out the writing task
2. write a few paragraphs for each chunk


## Utils

In [ ]:
# count the tokens utility
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    # encoding = tiktoken.get_encoding(encoding_name)
    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
    num_tokens = len(encoding.encode(string))
    return num_tokens

# num_tokens_from_string(text_body, "gpt-3.5-turbo")

In [ ]:
import textwrap
import os

def wrap_text(text: str, width: int = 120) -> str:
    """Wrap text to a specified width."""
    return '\n'.join(textwrap.wrap(text, width))

def write_string_to_file(filename: str, text: str) -> None:
    """Write a string to a file."""
    with open(filename, 'w') as f:
        f.write(text)

def extract_filename(path_string):
    """Extract filename without extension from a path string."""
    base_name = os.path.basename(path_string)  # Get the filename with extension
    file_name_without_ext = os.path.splitext(base_name)[0]  # Remove the extension
    return file_name_without_ext


## LangChain

In [ ]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain


#### System prompt

In [ ]:

context_template="You are a master writer that specializes in creating long 5,000 word articles. Please use all your expertise to approach this task. Output your content in markdown format and include titles and subtitles where relevant."


In [ ]:
system_message_prompt = SystemMessagePromptTemplate.from_template(context_template)


#### Human Prompt


In [ ]:

human_template="You are a master writer that specializes in creating long 5,000 word articles. Please use all your expertise to approach this task. Output your content in markdown format and include titles and subtitles where relevant. \n\n \
Please generate 15 questions . The questions should what the topic is about and allow you to flesh out each of these questions into a 500 word answer that covers all elements of the question. \n\n Topic: \n\n{topic_description} \
\n\n Once you have generated the 15 questions, think carefully on each of them and write an article that contains a Main title, subtitle for each sub topic and the 500 words (multiptle paragraphs) answer that addresses each of the questions. Don't phrase all sub headings as questions!! \n\n  Questions:\n\n Article: \n\n"


In [ ]:

human_message_prompt = HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            template=human_template,
            input_variables=["topic_description"],
        )
    )

chat_prompt_template = ChatPromptTemplate.from_messages([system_message_prompt,
                                                         human_message_prompt])

chat = ChatOpenAI(model_name="gpt-3.5-turbo-16k",
                  temperature=0.2)



In [ ]:
chain = LLMChain(llm=chat, prompt=chat_prompt_template)


## Full output

In [ ]:
TOPIC_DESCRIPTION = "The pros and cons of regulating AI research and development"

In [ ]:
%%time
from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:
    output = chain.run(TOPIC_DESCRIPTION)

    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost}")
    print(f"Calc Total Cost (USD): ${(cb.prompt_tokens/1000)*0.003 + (cb.completion_tokens/1000)*0.004}")

Total Tokens: 1072
Prompt Tokens: 169
Completion Tokens: 903
Total Cost (USD): $0.0
Calc Total Cost (USD): $0.004119
CPU times: user 374 ms, sys: 41.2 ms, total: 415 ms
Wall time: 1min 21s


In [ ]:
print(wrap_text(output))

# The Pros and Cons of Regulating AI Research and Development  ## Introduction  Artificial Intelligence (AI) has become
an integral part of our lives, from voice assistants to self-driving cars. As AI continues to advance, there is an
ongoing debate about the need for regulation in its research and development. On one hand, regulation can ensure ethical
and responsible use of AI, protecting individuals and society. On the other hand, excessive regulation may stifle
innovation and hinder progress. In this article, we will explore the pros and cons of regulating AI research and
development.  ## The Pros of Regulating AI Research and Development  ### 1. Ensuring Ethical Use of AI  Regulating AI
research and development can help ensure that AI systems are developed and used ethically. AI has the potential to
impact various aspects of our lives, including privacy, employment, and decision-making. By implementing regulations, we
can establish guidelines for developers and users to follow, pr

In [ ]:
write_string_to_file(f'Article-{TOPIC_DESCRIPTION}.txt', output)

In [ ]:
num_tokens_from_string(output, "gpt-3.5-turbo")

903

## Generate questions

#### Prompts


In [ ]:
context_template="You are a master writer that specializes in creating long 5,000 word articles. Please use all your expertise to approach this task. Output your content in markdown format and include titles and subtitles where relevant."

system_message_prompt = SystemMessagePromptTemplate.from_template(context_template)

human_template="Please generate 15 questions . The questions should what the topic is about and allow you to flesh out each of these questions into a 500 word answer that covers all elements of the question. \n\n Topic: \n\n{topic_description} \
\n\n  Questions:"

human_message_prompt = HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            template=human_template,
            input_variables=["topic_description"],
        )
    )

chat_prompt_template = ChatPromptTemplate.from_messages([system_message_prompt,
                                                         human_message_prompt])

chat_llm = ChatOpenAI(model_name="gpt-3.5-turbo",
                  temperature=0.2)

question_chain = LLMChain(llm=chat_llm, prompt=chat_prompt_template)

In [ ]:
TOPIC_DESCRIPTION = "The pros and cons of regulating AI research and development"

In [ ]:
%%time
from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:
    output = question_chain.run(TOPIC_DESCRIPTION)

    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost}")
    print(f"Calc Total Cost (USD): ${(cb.prompt_tokens/1000)*0.003 + (cb.completion_tokens/1000)*0.004}")

Total Tokens: 336
Prompt Tokens: 110
Completion Tokens: 226
Total Cost (USD): $0.0006720000000000001
Calc Total Cost (USD): $0.001234
CPU times: user 62.2 ms, sys: 8.36 ms, total: 70.6 ms
Wall time: 9.48 s


In [ ]:
print(wrap_text(output))

1. What is AI research and development, and why is it important? 2. What are the potential benefits of regulating AI
research and development? 3. What are the potential drawbacks of regulating AI research and development? 4. What are the
ethical considerations surrounding AI research and development? 5. What are the potential risks of unregulated AI
research and development? 6. What are the potential benefits of unregulated AI research and development? 7. What are the
current regulations surrounding AI research and development? 8. What are the potential consequences of implementing new
regulations on AI research and development? 9. What are the potential consequences of not implementing new regulations
on AI research and development? 10. What are the potential economic impacts of regulating AI research and development?
11. What are the potential societal impacts of regulating AI research and development? 12. What are the potential
environmental impacts of regulating AI research and dev

In [ ]:
write_string_to_file(f'Questions-{TOPIC_DESCRIPTION}.txt', output)

In [ ]:
num_tokens_from_string(output, "gpt-3.5-turbo")

226

In [ ]:
questions = output

## Generate the article

In [ ]:
context_template="You are a master writer that specializes in creating long 5,000 word articles. Please use all your expertise to approach this task. Output your content in markdown format and include titles and subtitles where relevant."

system_message_prompt = SystemMessagePromptTemplate.from_template(context_template)

human_template="Please take this list of questions and step by step go through each of them writing 4 paragraphs for each question. Include a subtitle for the question that rephrases the question. \n\n QUESTIONS: \n\n{questions} \
\n\n  ARTICLE:"

human_message_prompt = HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            template=human_template,
            input_variables=["questions"],
        )
    )

chat_prompt_template = ChatPromptTemplate.from_messages([system_message_prompt,
                                                         human_message_prompt])

chat_llm = ChatOpenAI(model_name="gpt-3.5-turbo-16k",
                  temperature=0.2)

question_chain = LLMChain(llm=chat_llm, prompt=chat_prompt_template)

In [ ]:
%%time
from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:
    output = question_chain.run(TOPIC_DESCRIPTION)

    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost}")
    print(f"Calc Total Cost (USD): ${(cb.prompt_tokens/1000)*0.003 + (cb.completion_tokens/1000)*0.004}")

Total Tokens: 1180
Prompt Tokens: 107
Completion Tokens: 1073
Total Cost (USD): $0.0
Calc Total Cost (USD): $0.004613
CPU times: user 413 ms, sys: 49.5 ms, total: 462 ms
Wall time: 1min 33s


In [ ]:
print(wrap_text(output))

## The Pros and Cons of Regulating AI Research and Development  ### Introduction  Artificial Intelligence (AI) has
become an integral part of our lives, from voice assistants on our smartphones to self-driving cars. As AI technology
continues to advance at an unprecedented pace, concerns about its potential risks and ethical implications have also
grown. This has led to a debate on whether there should be regulations in place to govern AI research and development.
In this article, we will explore the pros and cons of regulating AI research and development, weighing the potential
benefits against the potential drawbacks.  ### Pros of Regulating AI Research and Development  #### Ensuring Ethical Use
of AI  One of the main arguments in favor of regulating AI research and development is the need to ensure ethical use of
this powerful technology. AI has the potential to greatly impact society, and without proper regulations, there is a
risk of misuse or unintended consequences. By implement

In [ ]:
write_string_to_file(f'Questions-{TOPIC_DESCRIPTION}.txt', output)

In [ ]:
num_tokens_from_string(output, "gpt-3.5-turbo")

1073

In [ ]:
questions = output

## Generate the article

In [ ]:
context_template="You are a master writer that specializes in creating long 5,000 word articles. Please use all your expertise to approach this task. Output your content in markdown format and include titles and subtitles where relevant."

system_message_prompt = SystemMessagePromptTemplate.from_template(context_template)

human_template="Please take this list of questions and step by step go through each of them writing 4 paragraphs for each question. Include a subtitle for the question that rephrases the question. \n\n QUESTIONS: \n\n{questions} \
\n\n  ARTICLE:"

human_message_prompt = HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            template=human_template,
            input_variables=["questions"],
        )
    )

chat_prompt_template = ChatPromptTemplate.from_messages([system_message_prompt,
                                                         human_message_prompt])

chat_llm = ChatOpenAI(model_name="gpt-3.5-turbo-16k",
                  temperature=0.2)

article_chain = LLMChain(llm=chat_llm, prompt=chat_prompt_template)

In [ ]:
%%time
from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:
    output = article_chain.run(TOPIC_DESCRIPTION)

    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost}")
    print(f"Calc Total Cost (USD): ${(cb.prompt_tokens/1000)*0.003 + (cb.completion_tokens/1000)*0.004}")

Total Tokens: 1355
Prompt Tokens: 107
Completion Tokens: 1248
Total Cost (USD): $0.0
Calc Total Cost (USD): $0.005313
CPU times: user 500 ms, sys: 68.9 ms, total: 569 ms
Wall time: 1min 54s


In [ ]:
print(wrap_text(output))

## The Pros and Cons of Regulating AI Research and Development  ### Introduction  Artificial Intelligence (AI) has
emerged as one of the most transformative technologies of our time. With its potential to revolutionize industries and
improve our daily lives, it is no wonder that AI research and development (R&D) has gained significant attention.
However, as AI continues to advance, concerns about its ethical implications and potential risks have also arisen. This
has led to a debate on whether regulating AI R&D is necessary. In this article, we will explore the pros and cons of
regulating AI research and development, weighing the benefits and drawbacks of such regulations.  ### Pros of Regulating
AI Research and Development  #### Ensuring Ethical Development  One of the primary arguments in favor of regulating AI
research and development is the need to ensure ethical development. AI systems have the potential to make decisions that
can have significant consequences for individuals and 

In [ ]:
write_string_to_file(f'Article-{TOPIC_DESCRIPTION}.txt', output)

In [ ]:
num_tokens_from_string(output, "gpt-3.5-turbo")

1248